In [6]:
import numpy as np
import scipy
from CATE_utils import data_generator, regressor, clf, generate_EBM_CATE_learners, generate_CATE_learners, run_and_report_mse

In [56]:
n = 10000
d = 4
covariates_model = lambda d: np.random.binomial(1, .5, size=d)
propensity_model = lambda x: scipy.special.expit(x[0])
control_outcome_model = lambda x: x[0] + (x[0] + 1)*np.random.normal(0, 1, size=1)
treatment_effect_model = lambda x: -1 + 2 * x[0]
special_test_point = np.array([[1, 0, 0, 0], [0, 0, 0, 0]])
special_test_value = np.array([1, -1])


data1 = data_generator(n=n, 
                      d=d, 
                      covariates_model=covariates_model,
                      propensity_model=propensity_model,
                      control_outcome_model=control_outcome_model,
                      treatment_effect_model=treatment_effect_model,
                      special_test_point=special_test_point,
                      special_test_value=special_test_value,
                      seed = 1234
                      )

In [57]:
data = data1
X = data['X']
T = data['T']
y = data['Y']
X_test = data['X_test']
treatment_effect_test = data['treatment_effect_test']
y_test = data['Y_test']

P = data['P']
Y0 = data['Y0']
Y1 = data['Y1']
treatment_effect = data['treatment_effect']
special_test_point, special_test_value = data['special_test']

# T-Learner

In [70]:
from interpret.glassbox import DPExplainableBoostingClassifier, DPExplainableBoostingRegressor
from interpret.glassbox import ExplainableBoostingClassifier, ExplainableBoostingRegressor
from lightgbm import LGBMRegressor

epsilon = 4
delta = 1e-5

treatment_model = LGBMRegressor()
treatment_model.fit(X[T == 1], y[T == 1])

control_model = LGBMRegressor()
control_model.fit(X[T == 0], y[T == 0])

# treatment_model = DPExplainableBoostingRegressor(epsilon=epsilon, delta=delta, outer_bags=1, validation_size=0)
# treatment_model.fit(X[T == 1], y[T == 1])

# control_model = DPExplainableBoostingRegressor(epsilon=epsilon, delta=delta, outer_bags=1, validation_size=0)
# control_model.fit(X[T == 0], y[T == 0])

LGBMRegressor()

LGBMRegressor()

In [71]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(treatment_effect, treatment_model.predict(X) - control_model.predict(X))

# Doubly Robust Learner

In [78]:
# data splitting

from sklearn.model_selection import train_test_split

X1, X2, T1, T2, y1, y2 = train_test_split(X, T, y, test_size=0.5, random_state=42)
X11, X12, T11, T12, y11, y12 = train_test_split(X1, T1, y1, test_size=0.5, random_state=42)

In [79]:
epsilon = 4
delta = 1e-5

prop_model = DPExplainableBoostingClassifier(epsilon=epsilon, delta=delta, outer_bags=1, validation_size=0)
prop_model.fit(X11, T11)

treatment_model = DPExplainableBoostingRegressor(epsilon=epsilon, delta=delta, outer_bags=1, validation_size=0)
treatment_model.fit(X12[T12 == 1], y12[T12 == 1])

control_model = DPExplainableBoostingRegressor(epsilon=epsilon, delta=delta, outer_bags=1, validation_size=0)
control_model.fit(X12[T12 == 0], y12[T12 == 0])

Possible privacy violation: assuming min/max values per feature/target are public info.Pass a privacy schema with known public ranges to avoid this warning.


DPExplainableBoostingClassifier(epsilon=4,
                                feature_names=['feature_0001', 'feature_0002',
                                               'feature_0003', 'feature_0004'],
                                feature_types=['categorical', 'categorical',
                                               'categorical', 'categorical'],
                                outer_bags=1,
                                privacy_schema={0: (0, 1), 1: (0, 1), 2: (0, 1),
                                                3: (0, 1), 'target': (0, 1)},
                                validation_size=0)

DPExplainableBoostingRegressor(epsilon=4,
                               feature_names=['feature_0001', 'feature_0002',
                                              'feature_0003', 'feature_0004'],
                               feature_types=['categorical', 'categorical',
                                              'categorical', 'categorical'],
                               outer_bags=1,
                               privacy_schema={0: (0, 1), 1: (0, 1), 2: (0, 1),
                                               3: (0, 1),
                                               'target': (-4.0384670875703375,
                                                          7.954568064647068)},
                               validation_size=0)

DPExplainableBoostingRegressor(epsilon=4,
                               feature_names=['feature_0001', 'feature_0002',
                                              'feature_0003', 'feature_0004'],
                               feature_types=['categorical', 'categorical',
                                              'categorical', 'categorical'],
                               outer_bags=1,
                               privacy_schema={0: (0, 1), 1: (0, 1), 2: (0, 1),
                                               3: (0, 1),
                                               'target': (-3.875713300709113,
                                                          7.965690841101843)},
                               validation_size=0)

In [80]:
# Build CATE model

y_treatment_dr = treatment_model.predict(X2) + (y2 - treatment_model.predict(X2)) / (prop_model.predict_proba(X2)[:, 1]) * T2
y_control_dr = control_model.predict(X2) + (y2 - control_model.predict(X2)) / (prop_model.predict_proba(X2)[:, 1]) * (1 - T2)

phi_dr = y_treatment_dr - y_control_dr

cate_model = DPExplainableBoostingRegressor(epsilon=epsilon, delta=delta, outer_bags=1, validation_size=0)
cate_model.fit(X2, phi_dr)

DPExplainableBoostingRegressor(epsilon=4,
                               feature_names=['feature_0001', 'feature_0002',
                                              'feature_0003', 'feature_0004'],
                               feature_types=['categorical', 'categorical',
                                              'categorical', 'categorical'],
                               outer_bags=1,
                               privacy_schema={0: (0, 1), 1: (0, 1), 2: (0, 1),
                                               3: (0, 1),
                                               'target': (-10.188471306893566,
                                                          10.763838859789738)},
                               validation_size=0)

In [81]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(treatment_effect, cate_model.predict(X))

## Doubly Robust Linear Model

In [82]:
from diffprivlib.models import LogisticRegression, LinearRegression

epsilon = 4
delta = 1e-5

prop_model = LogisticRegression(epsilon=epsilon)
prop_model.fit(X11, T11)

treatment_model = LinearRegression(epsilon=epsilon)
treatment_model.fit(X12[T12 == 1], y12[T12 == 1])

control_model = LinearRegression(epsilon=epsilon)
control_model.fit(X12[T12 == 0], y12[T12 == 0])

Data norm has not been specified and will be calculated on the data provided.  This will result in additional privacy leakage. To ensure differential privacy and no additional privacy leakage, specify `data_norm` at initialisation.


LogisticRegression(accountant=BudgetAccountant(spent_budget=[(4, 0), (4, 0), (4, 0), (4, 0)]),
                   data_norm=2.0, epsilon=4)

Bounds parameters haven't been specified, so falling back to determining bounds from the data.
This will result in additional privacy leakage. To ensure differential privacy with no additional privacy loss, specify `bounds_X` and `bounds_y`.


LinearRegression(accountant=BudgetAccountant(spent_budget=[(4, 0), (4, 0), (4, 0), (4, 0), (4, 0)]),
                 bounds_X=(array([0., 0., 0., 0.]), array([1., 1., 1., 1.])),
                 bounds_y=(array([-4.03846709]), array([7.95456806])),
                 epsilon=4)

Bounds parameters haven't been specified, so falling back to determining bounds from the data.
This will result in additional privacy leakage. To ensure differential privacy with no additional privacy loss, specify `bounds_X` and `bounds_y`.


LinearRegression(accountant=BudgetAccountant(spent_budget=[(4, 0), (4, 0), (4, 0), (4, 0), (4, 0), ...]),
                 bounds_X=(array([0., 0., 0., 0.]), array([1., 1., 1., 1.])),
                 bounds_y=(array([-3.8757133]), array([7.96569084])),
                 epsilon=4)

In [84]:
# Build CATE model

y_treatment_dr = treatment_model.predict(X2) + (y2 - treatment_model.predict(X2)) / (prop_model.predict_proba(X2)[:, 1]) * T2
y_control_dr = control_model.predict(X2) + (y2 - control_model.predict(X2)) / (prop_model.predict_proba(X2)[:, 1]) * (1 - T2)

phi_dr = y_treatment_dr - y_control_dr

cate_model = LinearRegression(epsilon=epsilon)
cate_model.fit(X2, phi_dr)

Bounds parameters haven't been specified, so falling back to determining bounds from the data.
This will result in additional privacy leakage. To ensure differential privacy with no additional privacy loss, specify `bounds_X` and `bounds_y`.


LinearRegression(accountant=BudgetAccountant(spent_budget=[(4, 0), (4, 0), (4, 0), (4, 0), (4, 0), ...]),
                 bounds_X=(array([0., 0., 0., 0.]), array([1., 1., 1., 1.])),
                 bounds_y=(array([-9.13065457]), array([9.62811118])),
                 epsilon=4)

In [85]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(treatment_effect, cate_model.predict(X))